In [42]:
import json
import requests
import re
from os.path import join
from os import listdir
from datetime import datetime
import progressbar
import pandas as pd
import os
from pathlib import Path
import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords
import pymorphy2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np

In [2]:
# Parsing of post
def parse_post(post):
    output_post = {}
    output_post['id'] = post['id']
    output_post['url'] = post['url']
    output_post['dateRFC'] = post['dateRFC']
    output_post['date'] = post['date']
    output_post['title'] = post['title']
    output_post['intro'] = post['intro']
    text = ''
    
    # Собираем текст в одно целое
    for block in post['blocks']:
        if block['type'] == 'text' or block['type'] == 'quote':
            text += block['data']['text'] + ' '
        if block['type'] == 'list':
            text += ' '.join(block['data']['items']) + ' '
        if block['type'] == 'rawhtml':
            text += cleanhtml(block['data']['raw']) + ' '
            
    # Удаляем штуки которые мешают читать текст
    text = text.replace('[','', -1)
    text = text.replace(']','', -1)
    text = text.replace("\r", " ")
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("&nbsp", " ")
    text = text.replace("    ", " ")
    output_post['text'] = re.sub(r'\(http\S+\)', '', text, flags=re.MULTILINE)
    return output_post

def save_post_to_json(path, post):
    name = 'post' + str(post['id']) + '.json'
    with open(join(path, name), 'w', encoding='utf-8') as output:
        json.dump(post, output, indent=4, ensure_ascii=False)
        
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [3]:
bar = progressbar.ProgressBar(maxval=41000).start()
# 3290 - 59400 offset
for offset in range(4700, 41000, 50):
    req_url = 'https://api.tjournal.ru/v1.8/timeline/index/recent?count=50&offset=' + str(offset)
    request = requests.get(req_url)
    data = json.loads(request.text)
    bar.update(offset)
    for item in data['result']:
        post = parse_post(item)
        date = datetime.fromtimestamp(post['date'])
        if date.year < 2020 and date.year > 2013 and len(post['text']) > 400:
            save_post_to_json(join('D:\\some stuff\\tj', str(date.year)), post)
    
bar.finish()

100% |########################################################################|


In [24]:
folder_names = ['2014', '2015', '2016', '2017', '2018', '2019']

path_glob = 'D:\\some stuff\\tj'

rows = []
for folder in folder_names:
    path = join(path_glob, folder)
    posts_names = listdir(path)
    for post in posts_names:
        with open(join(path, post), encoding='utf-8') as f:
            r = json.load(f)
            rows.append(r)
            
df = pd.DataFrame.from_dict(rows, orient='columns')

In [25]:
df.shape

(26025, 7)

In [26]:
df.head()

,id,url,dateRFC,date,title,intro,text
0,48962,https://tjournal.ru/internet/48962-meme-2013,"Wed, 01 Jan 2014 12:24:26 +0400",1388564666,"Мемы, которые мы заслужили","Гарлем шейк, тверкинг, кошка Путина, озадаченн...","Гарлем шейк, тверкинг, кошка Путина, озадаченн..."
1,49173,https://tjournal.ru/internet/49173-kernes,"Wed, 19 Feb 2014 01:00:06 +0400",1392757206,Типичный харьковский мэр,Мэра Харькова Геннадия Кернеса сравнивают с Ка...,Мэра Харькова Геннадия Кернеса сравнивают с Ка...
2,49237,https://tjournal.ru/tech/49237-2013-in-gadgets,"Thu, 02 Jan 2014 17:12:51 +0400",1388668371,"Рак, лебедь и айфон",Главные гаджеты 2013 года,Главные гаджеты 2013 года В последние годы люб...
3,49242,https://tjournal.ru/tech/49242-iphone-nsa,"Wed, 01 Jan 2014 17:09:07 +0400",1388581747,АНБ хвасталось полным доступом к iPhone,В документах Агентства национальной безопаснос...,В документах Агентства национальной безопаснос...
4,49245,https://tjournal.ru/tech/49245-sea-skype,"Thu, 02 Jan 2014 02:45:56 +0400",1388616356,Электронная армия Сирии от имени Skype призвал...,"Участники кибергруппировки, называющей себя «э...","Участники кибергруппировки, называющей себя «э..."


In [27]:
with open('pickles_tj\\big_df.pickle', 'wb') as output:
    pickle.dump(df, output)

In [28]:
path_df = "pickles_tj\\big_df.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [29]:
url_stopwords_ru = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"


def get_text(url, encoding='utf-8', to_lower=True):
    url = str(url)
    if url.startswith('http'):
        r = requests.get(url)
        if not r.ok:
            r.raise_for_status()
        return r.text.lower() if to_lower else r.text
    elif os.path.exists(url):
        with open(url, encoding=encoding) as f:
            return f.read().lower() if to_lower else f.read()
    else:
        raise Exception('parameter [url] can be either URL or a filename')


def normalize_tokens(tokens):
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(tok)[0].normal_form for tok in tokens]


def remove_stopwords(tokens, stopwords=None, min_length=4):
    if not stopwords:
        return tokens
    stopwords = set(stopwords)
    tokens = [tok
              for tok in tokens
              if tok not in stopwords and len(tok) >= min_length]
    return tokens


def tokenize_n_lemmatize(
    text, stopwords=None, normalize=True, 
    regexp=r'(?u)\b\w{4,}\b'):
    words = [w for sent in sent_tokenize(text)
             for w in regexp_tokenize(sent, regexp)]
    if normalize:
        words = normalize_tokens(words)
    if stopwords:
        words = remove_stopwords(words, stopwords)
    return words

stopwords_ru = get_text(url_stopwords_ru).splitlines()

In [30]:
def proc_df(df):
    nrows = len(df)
    bar = progressbar.ProgressBar(maxval=nrows).start()
    text_parsed_list = []
    for row in range(0, nrows):
        bar.update(row)
        text = df.loc[row]['title'] + ' ' + df.loc[row]['text']
        text = text.replace('_', ' ')
        text_parsed_list.append(' '.join(tokenize_n_lemmatize(text)))
    df['text_parsed'] = text_parsed_list
    bar.finish()

In [31]:
proc_df(df)

100% |########################################################################|


In [32]:
df.head()

,id,url,dateRFC,date,title,intro,text,text_parsed
0,48962,https://tjournal.ru/internet/48962-meme-2013,"Wed, 01 Jan 2014 12:24:26 +0400",1388564666,"Мемы, которые мы заслужили","Гарлем шейк, тверкинг, кошка Путина, озадаченн...","Гарлем шейк, тверкинг, кошка Путина, озадаченн...",мема который заслужить гарлем шейк тверкинг ко...
1,49173,https://tjournal.ru/internet/49173-kernes,"Wed, 19 Feb 2014 01:00:06 +0400",1392757206,Типичный харьковский мэр,Мэра Харькова Геннадия Кернеса сравнивают с Ка...,Мэра Харькова Геннадия Кернеса сравнивают с Ка...,типичный харьковский мэр харьков геннадий керн...
2,49237,https://tjournal.ru/tech/49237-2013-in-gadgets,"Thu, 02 Jan 2014 17:12:51 +0400",1388668371,"Рак, лебедь и айфон",Главные гаджеты 2013 года,Главные гаджеты 2013 года В последние годы люб...,лебедь айфон главное гаджет 2013 год последний...
3,49242,https://tjournal.ru/tech/49242-iphone-nsa,"Wed, 01 Jan 2014 17:09:07 +0400",1388581747,АНБ хвасталось полным доступом к iPhone,В документах Агентства национальной безопаснос...,В документах Агентства национальной безопаснос...,хвастаться полный доступ iphone документ агент...
4,49245,https://tjournal.ru/tech/49245-sea-skype,"Thu, 02 Jan 2014 02:45:56 +0400",1388616356,Электронная армия Сирии от имени Skype призвал...,"Участники кибергруппировки, называющей себя «э...","Участники кибергруппировки, называющей себя «э...",электронный армия сирия имя skype призвать бой...


In [34]:
with open('pickles_tj\\parsed_big_df.pickle', 'wb') as output:
    pickle.dump(df, output)

In [103]:
with open('pickles_tj/parsed_big_df.pickle', 'rb') as data:
    df = pickle.load(data)

In [83]:
with open('models/best_rfc_fin.pickle', 'rb') as data:
    rfc_fin = pickle.load(data)
    
with open('models/best_rfc_inv.pickle', 'rb') as data:
    rfc_inv = pickle.load(data)

In [62]:
with open('Pickles/tfidf.pickle', 'rb') as data:
    tfidf = pickle.load(data)

In [80]:
# Parameter election
ngram_range = (1, 1)
min_df = 10
max_df = 1.
max_features = 5000

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

features = tfidf.fit_transform(df['text_parsed']).toarray()

# TF-IDF object
with open('pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)

In [81]:
with open('Pickles/tfidf.pickle', 'rb') as data:
    tfidf = pickle.load(data)
features = tfidf.transform(df['text_parsed']).toarray()

In [84]:
fin_pred = rfc_fin.predict(features)

In [85]:
inv_pred = rfc_inv.predict(features)

In [86]:
np.unique(fin_pred, return_counts=True)

(array([0, 1], dtype=int64), array([25752,   273], dtype=int64))

In [87]:
np.unique(inv_pred, return_counts=True)

(array([0], dtype=int64), array([26025], dtype=int64))

In [89]:
fin = [0] * 72
inv = [0] * 72
total = [0] * 72
for i in range(len(df)):
    date = datetime.fromtimestamp(df.loc[i]['date'])
    j = (date.year - 2014) * 12 + date.month - 1 
    total[j] += 1
    if fin_pred[i] == 1:
        fin[j] += 1
    if inv_pred[i] == 1:
        inv[j] += 1        

In [92]:
data = {'Total' : total, 'Fintech' : fin} 

In [93]:
out = pd.DataFrame(data) 

In [94]:
out.to_csv('tj.csv', sep=';')

In [117]:
tot_res = [0] * 72
fin_res = [0] * 72
inv_res = [0] * 72
words_tf_fin = ['финтех', 'криптовалюта', 'блокчейн', 'краудфандинг']
words_tf_fin_p = [('инвестиция','стартап'),('инвестиция', 'финтех'),('онлайн', 'банк'), ('финансовый', 'рынок')]
words_tf_inv = [('венчурный', 'капитал'), ('венчурный', 'финансирование')]
for i in range(len(df)):
    date = datetime.fromtimestamp(df.loc[i]['date'])
    j = (date.year - 2014) * 12 + date.month - 1 
    tot_res[j] += 1
    f = False
    for w in words_tf_fin:
        if df.loc[i]['text_parsed'].find(w) != -1:
            f = True
            fin_res[j] += 1
            break
    if f == False:    
        for w1, w2 in words_tf_fin_p:
            if df.loc[i]['text_parsed'].find(w1) != -1 and df.loc[i]['text_parsed'].find(w2) != -1:
                fin_res[j] += 1
                break
    for w1, w2 in words_tf_inv:
            if df.loc[i]['text_parsed'].find(w1) != -1 and df.loc[i]['text_parsed'].find(w2) != -1:
                inv_res[j] += 1
                break

In [118]:
data = {'Total' : tot_res, 'Fintech' : fin_res, 'Investment': inv_res} 

In [119]:
out = pd.DataFrame(data) 

In [120]:
out.to_csv('tj_key_words.csv', sep=';')